In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/quarantine-activity-US-Apr16-long.csv")

In [4]:
import numpy as np

In [5]:
from chartlib.utils import create_lockdown_type, split_into_list, str2emo

In [6]:
# START modified from _ingest_usa_quarantine_df
quarantine_csv = "data/quarantine-activity-US-Apr16-long.csv"
quarantine_df = pd.read_csv(quarantine_csv)
groupcol = 'Province_State'

quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)

quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])
# #TODO: this is where the columns get dropped
# quarantine_df = quarantine_df.groupby(['lockdown_date', 'Province_State']).agg({
#     'lockdown_type': lambda col: '; '.join(col),
#     'emoji_string': lambda col: ''.join(col),
#     'population_size': lambda col:col,
#     'Coverage.location': lambda col:col
# }).reset_index()

quarantine_df.loc[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine","emoji_string"]="t"#bugfix
# Breaking up emoji into separate rows for vertical stacking
quarantine_df.emoji_string = quarantine_df.emoji_string.apply(split_into_list)

quarantine_df = quarantine_df.explode(column='emoji_string')
quarantine_df['Coverage'] = quarantine_df.emoji_string.apply(
    lambda x: 'Statewide' if str(x).isupper() else 'Regional'
)
quarantine_df = quarantine_df.sort_values('Coverage', ascending=False)
quarantine_df.emoji_string = quarantine_df.emoji_string.str.lower()
quarantine_df['emoji'] = quarantine_df['emoji_string'].map(str2emo)
quarantine_df['event_index'] = quarantine_df.groupby(['Province_State', 'lockdown_date']).cumcount()

# # quarantine_cols = [
# #     groupcol, 'lockdown_date', 'lockdown_type', 'emoji', 'emoji_string', 'event_index', 'Coverage','Coverage.location'
# # ]
quarantine_cols = ['Province_State', 'Coverage', 'Coverage.location', 'lockdown_date','population_size','lockdown_type', 'emoji_string', 'emoji',
       'event_index']
quarantine_df = quarantine_df[quarantine_cols]

# # END modified from _ingest_usa_quarantine_df

In [7]:
quarantine_df_orig = pd.read_csv(quarantine_csv)

state_populations = quarantine_df_orig[quarantine_df_orig["Coverage.type"]=="State-wide"][["State","population_size"]].drop_duplicates()
# state_populations = state_populations.set_index(state_populations.State)
# state_populations = state_populations.drop(columns="State")
# state_populations_lookup= state_populations.population_size.to_dict()

In [8]:
state_populations = state_populations.rename(columns={'State': 'Province_State',"population_size":"state_population_size"})

In [9]:
#https://www.census.gov/quickfacts/PR
state_populations.loc[state_populations["Province_State"]=="Puerto Rico","state_population_size"] = 3725789

In [10]:
state_populations.state_population_size = state_populations.state_population_size.astype("int")

In [11]:
quarantine_df = quarantine_df.merge(state_populations)

In [12]:
severityScore = {'g':0.2, 's':0.5, 'r':0.4, 'n':0.7, 'l':1, 't':0.3, 'c':0.4, 'e':0.1}

In [13]:
quarantine_df[["emoji_string","emoji"]].drop_duplicates()

,emoji_string,emoji
0,e,⚠️
1,n,🏬
2,l,🏠
3,s,🎓
4,g,👨‍👩‍👧‍👦
15,r,🍔
156,t,💼
722,c,🛩


In [14]:
# quarantine_df[quarantine_df["population_size"].isna()]
quarantine_df = quarantine_df.dropna(subset=["population_size"])

In [15]:
quarantine_df.population_size = quarantine_df.population_size.str.replace(",","")

In [16]:
quarantine_df.population_size = quarantine_df.population_size.astype("int")

In [17]:
quarantine_df["pctStateAffected"]= quarantine_df["population_size"]/quarantine_df["state_population_size"]

In [18]:
assert all(quarantine_df[quarantine_df["pctStateAffected"]==1]["Coverage"]=="Statewide")

In [19]:
quarantine_df["severityScore"] = quarantine_df.emoji_string.apply(lambda x: severityScore[x])

In [20]:
quarantine_df["interventionFootprint"] = quarantine_df["pctStateAffected"]*quarantine_df["severityScore"]

In [21]:
quarantine_df.lockdown_date = quarantine_df.lockdown_date.str.replace("/","-")

In [22]:
# from chartlib import CovidChart, DaysSinceNumReached, days_between
# days_since = 20
# jhu_df = pd.read_csv('./data/jhu-data.csv')
# # grab us-specific
# jhu_df = jhu_df[(jhu_df.Country_Region == 'United States') & jhu_df.Province_State.notnull()]
# groupcol = 'Province_State'
# chart = CovidChart(
#     jhu_df,
#     groupcol=groupcol,
#     start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
#     ycol='interventionFootprint',
#     level='usa',
#     xcol='Date',
#     top_k_groups=30,
#     quarantine_df=quarantine_df
# )
# from datetime import datetime
# def first_alphabetic_group(df, groupcol):
#     return sorted(df[groupcol].unique())[0]
# EXTRA_DAYS_TO_INCLUDE = days_between('2020-04-28', datetime.now())
# chart.set_unfocused_opacity(0.05)
# chart = chart.set_ytitle('Number of Confirmed Cases (log scale)')
# chart = chart.set_xtitle(['Days since {} Confirmed'.format(days_since), '(Events prior to Day 0 not shown)'])
# chart.set_width(600).set_height(400)
# chart.set_xdomain((0, 47 + EXTRA_DAYS_TO_INCLUDE)).set_ydomain((days_since, 200000))
# chart.click_selection_init = first_alphabetic_group(chart._preprocess_df(), groupcol)

In [23]:
quarantine_df.lockdown_date = pd.to_datetime(quarantine_df.lockdown_date)

In [24]:
quarantine_df = quarantine_df[['Province_State', 'Coverage', 'Coverage.location', 'lockdown_date',
       'population_size', 'state_population_size', 'severityScore']]

In [25]:
state = "California"
qdf = quarantine_df[quarantine_df["Province_State"]==state]

In [26]:
qdf

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore
61,California,Statewide,California,2020-03-12,39512223,39512223,0.2
62,California,Statewide,California,2020-03-19,39512223,39512223,1.0
63,California,Statewide,California,2020-03-04,39512223,39512223,0.1
64,California,Regional,San Joaquin,2020-03-13,762148,39512223,0.5
65,California,Regional,San Luis Obispo,2020-03-13,283111,39512223,0.5
66,California,Regional,San Francisco,2020-03-17,881549,39512223,1.0
67,California,Regional,San Diego,2020-03-13,3338330,39512223,0.5
68,California,Regional,Sacramento,2020-03-13,1552058,39512223,0.5
69,California,Regional,Marin,2020-03-17,258826,39512223,1.0
70,California,Regional,Placer,2020-03-13,398329,39512223,0.5


In [27]:
# qdf[["Coverage.location","population_size"]].drop_duplicates().iloc[1:].sum()

In [28]:
# qdf[["Coverage.location","population_size"]].drop_duplicates()

In [29]:
# The counties with interventions in California is only 30% of the total state population size

In [30]:
statewideEvents = qdf[qdf["Coverage"]=="Statewide"]

In [31]:
def compute_state_replacements(state):
    df_state_replacements = []
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    statewideEvents = qdf[qdf["Coverage"]=="Statewide"]
    relevantCountiesInState= list(set(qdf[qdf["Province_State"]==state]["Coverage.location"]) - set([state]))
    # adding "Others" dummy state populated with statewide population - sum(relevant counties population)
    relevantCountiesInState= list(set(qdf[qdf["Province_State"]==state]["Coverage.location"]) - set([state]))
    relevant_county_population_lookup = qdf[qdf["Coverage.location"].isin(relevantCountiesInState)][["Coverage.location","population_size"]].drop_duplicates()
    state_population_size = state_populations[state_populations["Province_State"]==state].state_population_size.values[0]
    other_population = state_population_size - relevant_county_population_lookup.population_size.sum()
    qdf.loc[len(qdf)] = [state,"Regional","Others",np.nan,other_population,state_population_size,np.nan]
    relevantCountiesInState  = relevantCountiesInState + ["Others"]
    for statewideEvent in statewideEvents.iterrows(): 
    #     statewideLockdown_date = "2020-03-12"
    #     statewideSeverity = "0.1"
        statewideLockdown_date = statewideEvent[1].lockdown_date
        statewideSeverity = statewideEvent[1].severityScore
        for county in relevantCountiesInState: 
            clone = np.repeat(qdf[qdf["Coverage.location"]==county].iloc[0],1)
            clone.lockdown_date = statewideLockdown_date
            clone.severityScore = statewideSeverity
            df_state_replacements.append(clone)

    df_state_replacements = pd.DataFrame(df_state_replacements,columns=qdf.columns)
    return df_state_replacements 

In [32]:
# compute_state_replacements("California")

In [33]:
# qdf.sort_values("lockdown_date")

In [34]:
qdf = quarantine_df[quarantine_df["Province_State"]==state]
qdf_additional = compute_state_replacements(state)

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
qdf_result = pd.concat([qdf,qdf_additional])

03/13/2020: "On March 13, schools were closed in Marin, Sacramento, San Joaquin, San Luis Obispo, Santa Clara, Solano, Placer, and Contra Costa counties, as well as the Oakland, Antioch, Santa Cruz, Los Angeles Unified, Chaffey Unified, Etiwanda, Fontana Unified, Ontario-Montclair, Alta Loma Unified, San Diego, Los Alamitos Unified, and Washington Unified school districts."

In [69]:
qdf_keep

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax
61,California,Statewide,California,2020-03-12,39512223,39512223,0.2,0.2
64,California,Regional,San Joaquin,2020-03-13,762148,39512223,0.5,0.5
65,California,Regional,San Luis Obispo,2020-03-13,283111,39512223,0.5,0.5
67,California,Regional,San Diego,2020-03-13,3338330,39512223,0.5,0.5
68,California,Regional,Sacramento,2020-03-13,1552058,39512223,0.5,0.5
70,California,Regional,Placer,2020-03-13,398329,39512223,0.5,0.5
71,California,Regional,Marin,2020-03-13,258826,39512223,0.5,0.5
75,California,Regional,Contra Costa,2020-03-13,1153526,39512223,0.5,0.5
76,California,Regional,Santa Cruz County,2020-03-13,273213,39512223,0.5,0.5
77,California,Regional,Santa Clara,2020-03-13,1927852,39512223,0.5,0.5


In [158]:
qdf_keep_all=[]
for ldate in qdf.lockdown_date.unique():
    # ldate = "2020-03-13"
    qdf_before = qdf_result[qdf_result["lockdown_date"]<=ldate]
    # Compute the max severity for that location for the given day
    qdf_before["severityMax"] = qdf_before.groupby("Coverage.location")["severityScore"].transform(max)
    qdf_keep = qdf_before[qdf_before["severityScore"] == qdf_before["severityMax"]]
    qdf_keep["dateBefore"] = ldate
    qdf_keep = qdf_keep.drop_duplicates(subset="Coverage.location") # applicable only for the last date when severityMax=1 for multiple lockdown_dates, keep just one so that last entry doesn't exceed 1
    qdf_keep_all.append(qdf_keep)
    display(qdf_keep)
qdf_keep_all = pd.concat(qdf_keep_all)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
63,California,Statewide,California,2020-03-04,39512223,39512223,0.1,0.1,2020-03-04
64,California,Regional,San Joaquin,2020-03-04,762148,39512223,0.1,0.1,2020-03-04
66,California,Regional,San Francisco,2020-03-04,881549,39512223,0.1,0.1,2020-03-04
77,California,Regional,Santa Clara,2020-03-04,1927852,39512223,0.1,0.1,2020-03-04
72,California,Regional,Contra Costa,2020-03-04,1153526,39512223,0.1,0.1,2020-03-04
68,California,Regional,Sacramento,2020-03-04,1552058,39512223,0.1,0.1,2020-03-04
79,California,Regional,Solano,2020-03-04,447643,39512223,0.1,0.1,2020-03-04
70,California,Regional,Placer,2020-03-04,398329,39512223,0.1,0.1,2020-03-04
80,California,Regional,Antioch,2020-03-04,111535,39512223,0.1,0.1,2020-03-04
74,California,Regional,Alameda,2020-03-04,1671329,39512223,0.1,0.1,2020-03-04


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
61,California,Statewide,California,2020-03-12,39512223,39512223,0.2,0.2,2020-03-12
64,California,Regional,San Joaquin,2020-03-12,762148,39512223,0.2,0.2,2020-03-12
66,California,Regional,San Francisco,2020-03-12,881549,39512223,0.2,0.2,2020-03-12
77,California,Regional,Santa Clara,2020-03-12,1927852,39512223,0.2,0.2,2020-03-12
72,California,Regional,Contra Costa,2020-03-12,1153526,39512223,0.2,0.2,2020-03-12
68,California,Regional,Sacramento,2020-03-12,1552058,39512223,0.2,0.2,2020-03-12
79,California,Regional,Solano,2020-03-12,447643,39512223,0.2,0.2,2020-03-12
70,California,Regional,Placer,2020-03-12,398329,39512223,0.2,0.2,2020-03-12
80,California,Regional,Antioch,2020-03-12,111535,39512223,0.2,0.2,2020-03-12
74,California,Regional,Alameda,2020-03-12,1671329,39512223,0.2,0.2,2020-03-12


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
61,California,Statewide,California,2020-03-12,39512223,39512223,0.2,0.2,2020-03-13
64,California,Regional,San Joaquin,2020-03-13,762148,39512223,0.5,0.5,2020-03-13
65,California,Regional,San Luis Obispo,2020-03-13,283111,39512223,0.5,0.5,2020-03-13
67,California,Regional,San Diego,2020-03-13,3338330,39512223,0.5,0.5,2020-03-13
68,California,Regional,Sacramento,2020-03-13,1552058,39512223,0.5,0.5,2020-03-13
70,California,Regional,Placer,2020-03-13,398329,39512223,0.5,0.5,2020-03-13
71,California,Regional,Marin,2020-03-13,258826,39512223,0.5,0.5,2020-03-13
75,California,Regional,Contra Costa,2020-03-13,1153526,39512223,0.5,0.5,2020-03-13
76,California,Regional,Santa Cruz County,2020-03-13,273213,39512223,0.5,0.5,2020-03-13
77,California,Regional,Santa Clara,2020-03-13,1927852,39512223,0.5,0.5,2020-03-13


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
61,California,Statewide,California,2020-03-12,39512223,39512223,0.2,0.2,2020-03-17
64,California,Regional,San Joaquin,2020-03-13,762148,39512223,0.5,0.5,2020-03-17
65,California,Regional,San Luis Obispo,2020-03-13,283111,39512223,0.5,0.5,2020-03-17
66,California,Regional,San Francisco,2020-03-17,881549,39512223,1.0,1.0,2020-03-17
67,California,Regional,San Diego,2020-03-13,3338330,39512223,0.5,0.5,2020-03-17
68,California,Regional,Sacramento,2020-03-13,1552058,39512223,0.5,0.5,2020-03-17
69,California,Regional,Marin,2020-03-17,258826,39512223,1.0,1.0,2020-03-17
70,California,Regional,Placer,2020-03-13,398329,39512223,0.5,0.5,2020-03-17
72,California,Regional,Contra Costa,2020-03-17,1153526,39512223,1.0,1.0,2020-03-17
73,California,Regional,San Mateo,2020-03-17,766573,39512223,1.0,1.0,2020-03-17


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
62,California,Statewide,California,2020-03-19,39512223,39512223,1.0,1.0,2020-03-19
66,California,Regional,San Francisco,2020-03-17,881549,39512223,1.0,1.0,2020-03-19
69,California,Regional,Marin,2020-03-17,258826,39512223,1.0,1.0,2020-03-19
72,California,Regional,Contra Costa,2020-03-17,1153526,39512223,1.0,1.0,2020-03-19
73,California,Regional,San Mateo,2020-03-17,766573,39512223,1.0,1.0,2020-03-19
74,California,Regional,Alameda,2020-03-17,1671329,39512223,1.0,1.0,2020-03-19
78,California,Regional,Santa Clara,2020-03-17,1927852,39512223,1.0,1.0,2020-03-19
64,California,Regional,San Joaquin,2020-03-19,762148,39512223,1.0,1.0,2020-03-19
68,California,Regional,Sacramento,2020-03-19,1552058,39512223,1.0,1.0,2020-03-19
79,California,Regional,Solano,2020-03-19,447643,39512223,1.0,1.0,2020-03-19


In [159]:
qdf_keep[qdf_keep["Coverage"]=="Regional"]

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
66,California,Regional,San Francisco,2020-03-17,881549,39512223,1.0,1.0,2020-03-19
69,California,Regional,Marin,2020-03-17,258826,39512223,1.0,1.0,2020-03-19
72,California,Regional,Contra Costa,2020-03-17,1153526,39512223,1.0,1.0,2020-03-19
73,California,Regional,San Mateo,2020-03-17,766573,39512223,1.0,1.0,2020-03-19
74,California,Regional,Alameda,2020-03-17,1671329,39512223,1.0,1.0,2020-03-19
78,California,Regional,Santa Clara,2020-03-17,1927852,39512223,1.0,1.0,2020-03-19
64,California,Regional,San Joaquin,2020-03-19,762148,39512223,1.0,1.0,2020-03-19
68,California,Regional,Sacramento,2020-03-19,1552058,39512223,1.0,1.0,2020-03-19
79,California,Regional,Solano,2020-03-19,447643,39512223,1.0,1.0,2020-03-19
70,California,Regional,Placer,2020-03-19,398329,39512223,1.0,1.0,2020-03-19


In [160]:
qdf_keep[qdf_keep["Coverage"]=="Regional"].population_size.sum()/39512223

1.0

In [161]:
# ldate = "2020-03-13"
# qdf_before = qdf_result[qdf_result["lockdown_date"]<=ldate]
# # Compute the max severity for that location for the given day
# qdf_before["severityMax"] = qdf_before.groupby("Coverage.location")["severityScore"].transform(max)
# qdf_keep = qdf_before[qdf_before["severityScore"] == qdf_before["severityMax"]]

# qdf_keep

In [162]:
qdf_keep_all

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore
63,California,Statewide,California,2020-03-04,39512223,39512223,0.1,0.1,2020-03-04
64,California,Regional,San Joaquin,2020-03-04,762148,39512223,0.1,0.1,2020-03-04
66,California,Regional,San Francisco,2020-03-04,881549,39512223,0.1,0.1,2020-03-04
77,California,Regional,Santa Clara,2020-03-04,1927852,39512223,0.1,0.1,2020-03-04
72,California,Regional,Contra Costa,2020-03-04,1153526,39512223,0.1,0.1,2020-03-04
...,...,...,...,...,...,...,...,...,...
80,California,Regional,Antioch,2020-03-19,111535,39512223,1.0,1.0,2020-03-19
76,California,Regional,Santa Cruz County,2020-03-19,273213,39512223,1.0,1.0,2020-03-19
67,California,Regional,San Diego,2020-03-19,3338330,39512223,1.0,1.0,2020-03-19
65,California,Regional,San Luis Obispo,2020-03-19,283111,39512223,1.0,1.0,2020-03-19


In [163]:
qdf_keep_all["pctStateAffected"]= qdf_keep_all["population_size"]/qdf_keep_all["state_population_size"]
qdf_keep_all["interventionFootprint"] = qdf_keep_all["pctStateAffected"]*qdf_keep_all["severityScore"]

In [164]:
qdf_keep_all = qdf_keep_all[qdf_keep_all["Coverage"]=="Regional"]

In [165]:
qdf_keep_all

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,state_population_size,severityScore,severityMax,dateBefore,pctStateAffected,interventionFootprint
64,California,Regional,San Joaquin,2020-03-04,762148,39512223,0.1,0.1,2020-03-04,0.019289,0.001929
66,California,Regional,San Francisco,2020-03-04,881549,39512223,0.1,0.1,2020-03-04,0.022311,0.002231
77,California,Regional,Santa Clara,2020-03-04,1927852,39512223,0.1,0.1,2020-03-04,0.048791,0.004879
72,California,Regional,Contra Costa,2020-03-04,1153526,39512223,0.1,0.1,2020-03-04,0.029194,0.002919
68,California,Regional,Sacramento,2020-03-04,1552058,39512223,0.1,0.1,2020-03-04,0.039280,0.003928
...,...,...,...,...,...,...,...,...,...,...,...
80,California,Regional,Antioch,2020-03-19,111535,39512223,1.0,1.0,2020-03-19,0.002823,0.002823
76,California,Regional,Santa Cruz County,2020-03-19,273213,39512223,1.0,1.0,2020-03-19,0.006915,0.006915
67,California,Regional,San Diego,2020-03-19,3338330,39512223,1.0,1.0,2020-03-19,0.084489,0.084489
65,California,Regional,San Luis Obispo,2020-03-19,283111,39512223,1.0,1.0,2020-03-19,0.007165,0.007165


In [166]:
qdf_keep_all_result = qdf_keep_all.groupby("dateBefore").sum().reset_index()

In [167]:
qdf_keep_all_result

,dateBefore,population_size,state_population_size,severityScore,severityMax,pctStateAffected,interventionFootprint
0,2020-03-04,39512223,592683345,1.5,1.5,1.0,0.100000
1,2020-03-12,39512223,592683345,3.0,3.0,1.0,0.200000
2,2020-03-13,39512223,592683345,6.3,6.3,1.0,0.279772
3,2020-03-17,39512223,592683345,10.2,10.2,1.0,0.389249
4,2020-03-19,39512223,592683345,15.0,15.0,1.0,1.000000


In [168]:
import altair as alt
alt.Chart(qdf_keep_all_result).mark_line().encode(
    x= "dateBefore",
    y = "interventionFootprint"
)

alt.Chart(...)

In [32]:
# qdf.interventionFootprint.cumsum()


In [430]:
df_all = []
for state in quarantine_df.Province_State.unique():
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    qdf = qdf.groupby("lockdown_date").sum().reset_index()
    qdf = qdf.agg({'lockdown_date':lambda x: x,'interventionFootprint':'cumsum'})
    qdf.interventionFootprint = qdf.interventionFootprint.clip(0,1)
    qdf["State"]=state
    df_all.append(qdf)

df_all = pd.concat(df_all)

In [431]:
chart = alt.Chart(df_all).mark_line().encode(
        x = "lockdown_date",
        y = "interventionFootprint",
        facet=alt.Facet('State:O', columns=4),
    )
chart 

alt.Chart(...)